In [686]:
# libraries to use
import requests
import pandas as pd
import json
import io
import itertools
import datetime
import numpy as np
import math

In [2]:
# use these headers for every request to NBA.com
headers  = {
    'Connection': 'keep-alive',
    'Accept': 'application/json, text/plain, */*',
    'x-nba-stats-token': 'true',
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.130 Safari/537.36',
    'x-nba-stats-origin': 'stats',
    'Sec-Fetch-Site': 'same-origin',
    'Sec-Fetch-Mode': 'cors',
    'Referer': 'https://stats.nba.com/',
    'Accept-Encoding': 'gzip, deflate, br',
    'Accept-Language': 'en-US,en;q=0.9',
}

In [517]:
Season='2021-22'
PerMode='Totals'

# make a list of NBA measure types to loop through for a player
MeasureTypes = [
    'Base',
    'Advanced',
    'Misc',
    'Scoring',
    'Usage'
]

# make a list of NBA seasons to loop through later
season_list = [
    '1996-97',
    '1997-98',
    '1998-99',
    '1999-00',
    '2000-01',
    '2001-02',
    '2002-03',
    '2003-04',
    '2004-05',
    '2005-06',
    '2006-07',
    '2007-08',
    '2008-09',
    '2009-10',
    '2010-11',
    '2011-12',
    '2012-13',
    '2013-14',
    '2014-15',
    '2015-16',
    '2016-17',
    '2017-18',
    '2018-19',
    '2019-20',
    '2020-21',
    '2021-22',
    '2022-23',
]

team_abbrev = ['BOS', 'GSW', 'DET', 'WAS', 'ATL', 'NOP', 'CHI', 'TOR', 'MEM',
       'MIN', 'CHA', 'UTA', 'PHX', 'POR', 'MIL', 'LAC', 'SAS', 'BKN',
       'NYK', 'DEN', 'IND', 'MIA', 'CLE', 'DAL', 'PHI', 'HOU', 'OKC',
       'ORL', 'SAC', 'LAL']

In [425]:
# get general (Measure=Base) player total stats (PerMode=Totals) for 2021-22 season (Season=2021-22) - example URL request
# get PLAYER_ID, PLAYER_NAME, GP, MIN
players_stats_url = 'https://stats.nba.com/stats/leaguedashplayerstats?College=&Conference=&Country=&DateFrom=&DateTo=&Division=&DraftPick=&DraftYear=&GameScope=&GameSegment=&Height=&LastNGames=0&LeagueID=00&Location=&MeasureType=Base&Month=0&OpponentTeamID=0&Outcome=&PORound=0&PaceAdjust=N&PerMode='+PerMode+'&Period=0&PlayerExperience=&PlayerPosition=&PlusMinus=N&Rank=N&Season='+Season+'&SeasonSegment=&SeasonType=Regular+Season&ShotClockRange=&StarterBench=&TeamID=0&TwoWay=0&VsConference=&VsDivision=&Weight='
players_stats_response = requests.get(url=players_stats_url, headers=headers).json()
players_stats_headers = players_stats_response['resultSets'][0]['headers']
players_stats_data = players_stats_response['resultSets'][0]['rowSet']
players_stats_df = pd.DataFrame(data=players_stats_data, columns=players_stats_headers)

In [426]:
player_ids_2021_22 = players_stats_df[['PLAYER_ID', 'PLAYER_NAME','GP','MIN']]
player_ids_2021_22

,PLAYER_ID,PLAYER_NAME,GP,MIN
0,203932,Aaron Gordon,75,2375.418333
1,1630565,Aaron Henry,6,16.983333
2,1628988,Aaron Holiday,63,1020.750000
3,1630174,Aaron Nesmith,52,573.878333
4,1630598,Aaron Wiggins,50,1208.750000
...,...,...,...,...
600,203897,Zach LaVine,67,2328.078333
601,1630285,Zavier Simpson,4,174.083333
602,1630192,Zeke Nnaji,41,697.608333
603,1630533,Ziaire Williams,62,1346.466667


In [429]:
mpg_list = []

for x in range(0, player_ids_2021_22.shape[0]):
    mpg = player_ids_2021_22.iloc[x]['MIN']/player_ids_2021_22.iloc[x]['GP']
    mpg_list.append(round(mpg,1))

player_ids_2021_22.loc[:,'MPG'] = mpg_list

C:\Users\Reed\AppData\Local\Temp\ipykernel_7540\553556145.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  player_ids_2021_22.loc[:,'MPG'] = mpg_list


In [430]:
player_ids_2021_22

,PLAYER_ID,PLAYER_NAME,GP,MIN,MPG
0,203932,Aaron Gordon,75,2375.418333,31.7
1,1630565,Aaron Henry,6,16.983333,2.8
2,1628988,Aaron Holiday,63,1020.750000,16.2
3,1630174,Aaron Nesmith,52,573.878333,11.0
4,1630598,Aaron Wiggins,50,1208.750000,24.2
...,...,...,...,...,...
600,203897,Zach LaVine,67,2328.078333,34.7
601,1630285,Zavier Simpson,4,174.083333,43.5
602,1630192,Zeke Nnaji,41,697.608333,17.0
603,1630533,Ziaire Williams,62,1346.466667,21.7


In [443]:
high_mpg_players_2021_22_df = player_ids_2021_22.loc[(player_ids_2021_22['MIN'] >= 500) & (player_ids_2021_22['MPG'] >= 30)]

In [444]:
high_mpg_players_2021_22_df

,PLAYER_ID,PLAYER_NAME,GP,MIN,MPG
0,203932,Aaron Gordon,75,2375.418333,31.7
21,203952,Andrew Wiggins,73,2329.513333,31.9
23,203076,Anthony Davis,40,1404.248333,35.1
24,1630162,Anthony Edwards,72,2465.993333,34.2
34,1628389,Bam Adebayo,56,1825.261667,32.6
...,...,...,...,...,...
571,1629639,Tyler Herro,66,2151.011667,32.6
574,1630169,Tyrese Haliburton,77,2694.131667,35.0
575,1630178,Tyrese Maxey,75,2650.356667,35.3
591,203115,Will Barton,71,2276.463333,32.1


In [445]:
high_mpg_players_2021_22_df.to_csv('players_mpg_2021_22_30mpg.csv', index=False)

In [451]:
# example NBA.com 2021-21 (Season=2021-22) players bio (https://stats.nba.com/stats/leaguedashplayerbiostats):
# use for AGE, PLAYER_HEIGHT_INCHES, PLAYER_WEIGHT
plyrs_bio_url = 'https://stats.nba.com/stats/leaguedashplayerbiostats?College=&Conference=&Country=&DateFrom=&DateTo=&Division=&DraftPick=&DraftYear=&GameScope=&GameSegment=&Height=&LastNGames=0&LeagueID=00&Location=&Month=0&OpponentTeamID=0&Outcome=&PORound=0&PerMode=PerGame&Period=0&PlayerExperience=&PlayerPosition=&Season='+Season+'&SeasonSegment=&SeasonType=Regular%20Season&ShotClockRange=&StarterBench=&TeamID=0&VsConference=&VsDivision=&Weight='
plyrs_bio_response = requests.get(url=plyrs_bio_url, headers=headers).json()
plyrs_bio_headers = plyrs_bio_response['resultSets'][0]['headers']
plyrs_bio_data = plyrs_bio_response['resultSets'][0]['rowSet']
plyrs_bio_df = pd.DataFrame(data=plyrs_bio_data, columns=plyrs_bio_headers)
plyrs_bio_headers

['PLAYER_ID',
 'PLAYER_NAME',
 'TEAM_ID',
 'TEAM_ABBREVIATION',
 'AGE',
 'PLAYER_HEIGHT',
 'PLAYER_HEIGHT_INCHES',
 'PLAYER_WEIGHT',
 'COLLEGE',
 'COUNTRY',
 'DRAFT_YEAR',
 'DRAFT_ROUND',
 'DRAFT_NUMBER',
 'GP',
 'PTS',
 'REB',
 'AST',
 'NET_RATING',
 'OREB_PCT',
 'DREB_PCT',
 'USG_PCT',
 'TS_PCT',
 'AST_PCT']

In [538]:
# example NBA.com 2021-21 (Season=2021-22) player index (https://stats.nba.com/stats/playergamelogs/playerindex):
# do we need anything...? Just get from bio?
plyrs_idx_url = 'https://stats.nba.com/stats/playerindex?College=&Country=&DraftPick=&DraftRound=&DraftYear=&Height=&Historical=1&LeagueID=00&Season='+Season+'&SeasonType=Regular%20Season&TeamID=0&Weight='
plyrs_idx_response = requests.get(url=plyrs_idx_url, headers=headers).json()
plyrs_idx_headers = plyrs_idx_response['resultSets'][0]['headers']
plyrs_idx_data = plyrs_idx_response['resultSets'][0]['rowSet']
plyrs_idx_df = pd.DataFrame(data=plyrs_idx_data, columns=plyrs_idx_headers)
plyrs_idx_headers

['PERSON_ID',
 'PLAYER_LAST_NAME',
 'PLAYER_FIRST_NAME',
 'PLAYER_SLUG',
 'TEAM_ID',
 'TEAM_SLUG',
 'IS_DEFUNCT',
 'TEAM_CITY',
 'TEAM_NAME',
 'TEAM_ABBREVIATION',
 'JERSEY_NUMBER',
 'POSITION',
 'HEIGHT',
 'WEIGHT',
 'COLLEGE',
 'COUNTRY',
 'DRAFT_YEAR',
 'DRAFT_ROUND',
 'DRAFT_NUMBER',
 'ROSTER_STATUS',
 'PTS',
 'REB',
 'AST',
 'STATS_TIMEFRAME',
 'FROM_YEAR',
 'TO_YEAR']

In [453]:
# PlayerID = 2544 # leBron James
# PlayerID = 202695 # kawhi leoard - lots of time off
# PlayerID = 201935 # James Harden - had 2021-22 trade BRK -> PHI
PlayerID = 1629029 # Luka Doncic

In [456]:
# get game logs for PlayerID from NBA.com (MeasureType=Base,Advanced,Misc,Scoring,Usage)
def get_game_logs_df(PlayerID, MeasureType, Season, PerMode):
    glogs_url = 'https://stats.nba.com/stats/playergamelogs?DateFrom=&DateTo=&GameSegment=&LastNGames=0&LeagueID=00&Location=&MeasureType='+MeasureType+'&Month=0&OppTeamID=0&OpponentTeamID=0&Outcome=&PORound=0&PaceAdjust=N&PerMode='+PerMode+'&Period=0&PlayerID='+str(PlayerID)+'&PlusMinus=N&Rank=N&Season='+Season+'&SeasonSegment=&SeasonType=Regular%20Season&ShotClockRange=&TeamID=&VsConference=&VsDivision='
    glogs_response = requests.get(url=glogs_url, headers=headers).json()
    glogs_headers = glogs_response['resultSets'][0]['headers']
    glogs_data = glogs_response['resultSets'][0]['rowSet']
    glogs_df = pd.DataFrame(data=glogs_data, columns=glogs_headers)
    return glogs_df

In [457]:
# get player game logs for PlayerID from NBA.com for all measure types (MeasureType=Base,Advanced,Misc,Scoring,Usage)
# PlayerID = 1629029 # Luka Doncic
# Season='2021-22'
# PerMode='Totals'
plyr_base_df = get_game_logs_df(PlayerID,'Base',Season, PerMode)
plyr_advanced_df = get_game_logs_df(PlayerID,'Advanced',Season, PerMode)
plyr_misc_df = get_game_logs_df(PlayerID,'Misc',Season, PerMode)
plyr_scor_df = get_game_logs_df(PlayerID,'Scoring',Season, PerMode)
plyr_usage_df = get_game_logs_df(PlayerID,'Usage',Season, PerMode)

In [502]:
# get subset of attributes from each game log measure

plyr_base_headers_subset = ['SEASON_YEAR',
 'PLAYER_ID',
 'PLAYER_NAME',
 'TEAM_ID',
 'TEAM_ABBREVIATION',
 'TEAM_NAME',
 'GAME_ID',
 'GAME_DATE',
 'MATCHUP',
 'WL',
 'MIN',
 'FGM',
 'FGA',
 'FG3M',
 'FG3A',
 'FTM',
 'FTA',
 'OREB',
 'DREB',
 'AST',
 'TOV',
 'STL',
 'BLK',
 'PF',
 'PFD',
 'PTS']

plyr_advanced_headers_subset = ['USG_PCT',
 'PIE',
 'POSS']

plyr_misc_headers_subset= [ 'PTS_PAINT']

plyr_scor_headers_subset = ['PCT_PTS_2PT_MR']

plyr_base_df_subset = plyr_base_df[plyr_base_headers_subset]
plyr_advanced_df_subset = plyr_advanced_df[plyr_advanced_headers_subset]
plyr_misc_df_subset = plyr_misc_df[plyr_misc_headers_subset]
plyr_scor_df_subset = plyr_scor_df[plyr_scor_headers_subset]

In [503]:
# put all game log subsets for player into one dataframe
plyr_box_stats_df = pd.concat([plyr_base_df_subset, 
                               plyr_advanced_df_subset, 
                               plyr_misc_df_subset, 
                               plyr_scor_df_subset], axis=1)

num_box_stats = plyr_box_stats_df.shape[0]

player_bio = plyrs_bio_df.loc[plyrs_bio_df['PLAYER_ID'] == PlayerID]

# cannot find player birthday info on NBA.com
# subtract age by one when going back single NBA season; this is the work-around for now
# As of 3/2/2023: for 2021-22 season, substract 1 from age at NBA.com
player_age = [int(player_bio.iloc[0]['AGE'])-1] * num_box_stats
plyr_box_stats_df['AGE'] = player_age

player_height = [player_bio.iloc[0]['PLAYER_HEIGHT_INCHES']] * num_box_stats
plyr_box_stats_df['PLAYER_HEIGHT_INCHES'] = player_height

player_weight = [player_bio.iloc[0]['PLAYER_WEIGHT']] * num_box_stats
plyr_box_stats_df['PLAYER_WEIGHT'] = player_weight

In [505]:
# calculate field goals made mid-range and in the paint
fg_attr = ['FGM','FGA','FG3M','FG3A','FTM','FTA','PTS','PTS_PAINT','PCT_PTS_2PT_MR']

fgm_paint_list = []
fgm_mr_list = []
for x in range(0, num_box_stats):
    game_stats = plyr_box_stats_df.iloc[x][fg_attr]
    fgm_paint = game_stats['PTS_PAINT']/2
    fgm_2p = game_stats['FGM'] - game_stats['FG3M']
    fgm_mr = fgm_2p - fgm_paint
    fgm_paint_list.append(int(fgm_paint))
    fgm_mr_list.append(int(fgm_mr))

# note: cannot find 2PT Mid-range FG attempts in player box scores at NBA.com;
# will use 2PT mid-range and in-the-paint made FG for now
plyr_box_stats_df['FGM_PAINT'] = fgm_paint_list
plyr_box_stats_df['FGM_2PT_MR'] = fgm_mr_list


In [530]:
plyr_box_stats_df

,SEASON_YEAR,PLAYER_ID,PLAYER_NAME,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,...,PW_TOV,PW_STL,PW_BLK,PW_POS,PW_PF,PW_PFD,PW_PTS_PAINT,PW_PTS_2PT_MR,PW_PIE_AVG,PW_USG_PCT_AVG
0,2021-22,1629029,Luka Doncic,1610612742,DAL,Dallas Mavericks,0022101219,2022-04-10T00:00:00,DAL vs. SAS,W,...,17,4,1,210,6,8.000000,30,0.140,0.216667,0.431333
1,2021-22,1629029,Luka Doncic,1610612742,DAL,Dallas Mavericks,0022101209,2022-04-08T00:00:00,DAL vs. POR,W,...,17,4,1,222,9,7.666667,28,0.307,0.160333,0.417000
2,2021-22,1629029,Luka Doncic,1610612742,DAL,Dallas Mavericks,0022101190,2022-04-06T00:00:00,DAL @ DET,W,...,14,4,3,229,6,6.333333,38,0.287,0.192000,0.390333
3,2021-22,1629029,Luka Doncic,1610612742,DAL,Dallas Mavericks,0022101167,2022-04-03T00:00:00,DAL @ MIL,W,...,12,2,3,270,8,6.250000,54,0.346,0.220000,0.390000
4,2021-22,1629029,Luka Doncic,1610612742,DAL,Dallas Mavericks,0022101152,2022-04-01T00:00:00,DAL @ WAS,L,...,10,3,2,270,5,6.000000,48,0.346,0.229750,0.358500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60,2021-22,1629029,Luka Doncic,1610612742,DAL,Dallas Mavericks,0022100075,2021-10-29T00:00:00,DAL @ DEN,L,...,16,2,3,220,9,5.333333,40,0.456,0.168667,0.356000
61,2021-22,1629029,Luka Doncic,1610612742,DAL,Dallas Mavericks,0022100069,2021-10-28T00:00:00,DAL vs. SAS,W,...,14,3,2,228,7,6.333333,28,0.678,0.179667,0.329667
62,2021-22,1629029,Luka Doncic,1610612742,DAL,Dallas Mavericks,0022100052,2021-10-26T00:00:00,DAL vs. HOU,W,...,10,1,1,153,4,6.000000,18,0.370,0.154500,0.318000
63,2021-22,1629029,Luka Doncic,1610612742,DAL,Dallas Mavericks,0022100029,2021-10-23T00:00:00,DAL @ TOR,W,...,5,1,0,77,1,6.000000,4,0.222,0.136000,0.276000


In [531]:
game_dates = plyr_box_stats_df['GAME_DATE'].values.tolist()
game_dates_obj = [datetime.datetime.strptime(date, '%Y-%m-%dT%H:%M:%S') for date in game_dates]
# add converted python GAME_DATE_OBJ datetime64 object to plyr_box_stats_df
plyr_box_stats_df.loc[:,'GAME_DATE_OBJ']  = game_dates_obj

In [532]:
plyr_box_stats_df

,SEASON_YEAR,PLAYER_ID,PLAYER_NAME,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,...,PW_STL,PW_BLK,PW_POS,PW_PF,PW_PFD,PW_PTS_PAINT,PW_PTS_2PT_MR,PW_PIE_AVG,PW_USG_PCT_AVG,GAME_DATE_OBJ
0,2021-22,1629029,Luka Doncic,1610612742,DAL,Dallas Mavericks,0022101219,2022-04-10T00:00:00,DAL vs. SAS,W,...,4,1,210,6,8.000000,30,0.140,0.216667,0.431333,2022-04-10
1,2021-22,1629029,Luka Doncic,1610612742,DAL,Dallas Mavericks,0022101209,2022-04-08T00:00:00,DAL vs. POR,W,...,4,1,222,9,7.666667,28,0.307,0.160333,0.417000,2022-04-08
2,2021-22,1629029,Luka Doncic,1610612742,DAL,Dallas Mavericks,0022101190,2022-04-06T00:00:00,DAL @ DET,W,...,4,3,229,6,6.333333,38,0.287,0.192000,0.390333,2022-04-06
3,2021-22,1629029,Luka Doncic,1610612742,DAL,Dallas Mavericks,0022101167,2022-04-03T00:00:00,DAL @ MIL,W,...,2,3,270,8,6.250000,54,0.346,0.220000,0.390000,2022-04-03
4,2021-22,1629029,Luka Doncic,1610612742,DAL,Dallas Mavericks,0022101152,2022-04-01T00:00:00,DAL @ WAS,L,...,3,2,270,5,6.000000,48,0.346,0.229750,0.358500,2022-04-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60,2021-22,1629029,Luka Doncic,1610612742,DAL,Dallas Mavericks,0022100075,2021-10-29T00:00:00,DAL @ DEN,L,...,2,3,220,9,5.333333,40,0.456,0.168667,0.356000,2021-10-29
61,2021-22,1629029,Luka Doncic,1610612742,DAL,Dallas Mavericks,0022100069,2021-10-28T00:00:00,DAL vs. SAS,W,...,3,2,228,7,6.333333,28,0.678,0.179667,0.329667,2021-10-28
62,2021-22,1629029,Luka Doncic,1610612742,DAL,Dallas Mavericks,0022100052,2021-10-26T00:00:00,DAL vs. HOU,W,...,1,1,153,4,6.000000,18,0.370,0.154500,0.318000,2021-10-26
63,2021-22,1629029,Luka Doncic,1610612742,DAL,Dallas Mavericks,0022100029,2021-10-23T00:00:00,DAL @ TOR,W,...,1,0,77,1,6.000000,4,0.222,0.136000,0.276000,2021-10-23


In [507]:
def get_games_past_week(this_game, plyr_box_stats_df):
    # for a given game_date find the week period before
    game_date_str = this_game['GAME_DATE']
    # parse game_date into python date object
    game_date = datetime.datetime.strptime(game_date_str, '%Y-%m-%dT%H:%M:%S')
    time_delta_yesterday = datetime.timedelta(days = 1)
    time_delta_week_ago = datetime.timedelta(days = 7)
    date_yesterday = game_date - time_delta_yesterday
    date_week_ago = game_date - time_delta_week_ago

    # convert game_date to python date objects
    game_dates = plyr_box_stats_df['GAME_DATE'].values.tolist()
    game_dates_obj = [datetime.datetime.strptime(date, '%Y-%m-%dT%H:%M:%S') for date in game_dates]

    # create temp df of only game_id and game_date
    game_dates_df = plyr_box_stats_df[['GAME_ID','GAME_DATE']]

    # add converted python game_date object to temp df
    game_dates_df.loc[:,'GAME_DATE_OBJ']  = game_dates_obj
    
    # find games that lie between date_yesterday and date_week_ago
    game_dates_past_week = game_dates_df.loc[(game_dates_df['GAME_DATE_OBJ'] <= date_yesterday) & (game_dates_df['GAME_DATE_OBJ'] >= date_week_ago)]
    game_ids_past_week = game_dates_past_week['GAME_ID'].values.tolist()
    games_past_week = plyr_box_stats_df.loc[plyr_box_stats_df['GAME_ID'].isin(game_ids_past_week)]
    
    return games_past_week

In [508]:
'''
Compile box stats containing stat totals from the past week (7 days before) of game_date for the following: 
Total minutes played - pw_min
Total games played - pw_games
2-point field goal attempts - pw_fg2a
3-point field goal attempts - pw_gf3a
Blocks - pw_blk
Rebounds - pw_reb (need offensive/defensive rebounds…?)
Personal fouls - pw_pf
Assists - pw_ast
'''

'\nCompile box stats containing stat totals from the past week (7 days before) of game_date for the following: \nTotal minutes played - pw_min\nTotal games played - pw_games\n2-point field goal attempts - pw_fg2a\n3-point field goal attempts - pw_gf3a\nBlocks - pw_blk\nRebounds - pw_reb (need offensive/defensive rebounds…?)\nPersonal fouls - pw_pf\nAssists - pw_ast\n'

In [509]:
pw_mins = []
pw_gamess = []
pw_fg3as = []
pw_fg2as = []
pw_orebs = []
pw_drebs = []
pw_asts = []
pw_tovs = []
pw_stls = []
pw_blks = []
pw_posss = []
pw_pfs = []
pw_pfds = []
pw_pts_paints = []
pw_pts_2pt_mrs = []
pw_pie_avg = []
pw_usg_pct_avg = []

In [510]:
# iterate through all games in player box stats and calculate totals/avgs for games the week before
for index, this_game in plyr_box_stats_df.iterrows():
    games_week_before_df = get_games_week_before(this_game, plyr_box_stats_df)
    pw_min = games_week_before_df['MIN'].sum()
    pw_games = games_week_before_df.shape[0]
    pw_fg3a = games_week_before_df['FG3A'].sum()
    pw_fg2a = games_week_before_df['FGA'].sum() - pw_fg3a
    pw_oreb = games_week_before_df['OREB'].sum()
    pw_dreb = games_week_before_df['DREB'].sum()
    pw_ast = games_week_before_df['AST'].sum()
    pw_tov = games_week_before_df['TOV'].sum()
    pw_stl = games_week_before_df['STL'].sum()
    pw_blk = games_week_before_df['BLK'].sum()
    pw_poss = games_week_before_df['POSS'].sum()
    pw_pf = games_week_before_df['PF'].sum()
    pw_pfd = games_week_before_df['PFD'].mean()
    pw_pts_paint = games_week_before_df['PTS_PAINT'].sum()
    pw_pts_2pt_mr = games_week_before_df['PCT_PTS_2PT_MR'].sum()
    pw_pie = games_week_before_df['PIE'].mean()
    pw_usg_pct = games_week_before_df['USG_PCT'].mean()
    
    pw_mins.append(pw_min)
    pw_gamess.append(pw_games)
    pw_fg3as.append(pw_fg3a)
    pw_fg2as.append(pw_fg2a)
    pw_orebs.append(pw_oreb)
    pw_drebs.append(pw_dreb)
    pw_asts.append(pw_ast)
    pw_tovs.append(pw_tov)
    pw_stls.append(pw_stl)
    pw_blks.append(pw_blk)
    pw_posss.append(pw_poss)
    pw_pfs.append(pw_pf)
    pw_pfds.append(pw_pfd)
    pw_pts_paints.append(pw_pts_paint)
    pw_pts_2pt_mrs.append(pw_pts_2pt_mr)
    pw_pie_avg.append(pw_pie)
    pw_usg_pct_avg.append(pw_usg_pct)

C:\Users\Reed\AppData\Local\Temp\ipykernel_7540\3095134818.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  game_dates_df.loc[:,'GAME_DATE_OBJ']  = game_dates_obj
C:\Users\Reed\AppData\Local\Temp\ipykernel_7540\3095134818.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  game_dates_df.loc[:,'GAME_DATE_OBJ']  = game_dates_obj
C:\Users\Reed\AppData\Local\Temp\ipykernel_7540\3095134818.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_i

C:\Users\Reed\AppData\Local\Temp\ipykernel_7540\3095134818.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  game_dates_df.loc[:,'GAME_DATE_OBJ']  = game_dates_obj
C:\Users\Reed\AppData\Local\Temp\ipykernel_7540\3095134818.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  game_dates_df.loc[:,'GAME_DATE_OBJ']  = game_dates_obj
C:\Users\Reed\AppData\Local\Temp\ipykernel_7540\3095134818.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_i

C:\Users\Reed\AppData\Local\Temp\ipykernel_7540\3095134818.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  game_dates_df.loc[:,'GAME_DATE_OBJ']  = game_dates_obj
C:\Users\Reed\AppData\Local\Temp\ipykernel_7540\3095134818.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  game_dates_df.loc[:,'GAME_DATE_OBJ']  = game_dates_obj
C:\Users\Reed\AppData\Local\Temp\ipykernel_7540\3095134818.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_i

In [511]:
plyr_box_stats_df.loc[:,'PW_MIN'] = pw_mins
plyr_box_stats_df.loc[:,'PW_GAMES'] = pw_gamess
plyr_box_stats_df.loc[:,'PW_GAMES'] = pw_gamess
plyr_box_stats_df.loc[:,'PW_FG3A'] = pw_fg3as
plyr_box_stats_df.loc[:,'PW_FG2A'] = pw_fg2as
plyr_box_stats_df.loc[:,'PW_OREB'] = pw_orebs
plyr_box_stats_df.loc[:,'PW_DREB'] = pw_drebs
plyr_box_stats_df.loc[:,'PW_ASTS'] = pw_asts
plyr_box_stats_df.loc[:,'PW_TOV'] = pw_tovs
plyr_box_stats_df.loc[:,'PW_STL'] = pw_stls
plyr_box_stats_df.loc[:,'PW_BLK'] = pw_blks
plyr_box_stats_df.loc[:,'PW_POS'] = pw_posss
plyr_box_stats_df.loc[:,'PW_PF'] = pw_pfs
plyr_box_stats_df.loc[:,'PW_PFD'] = pw_pfds
plyr_box_stats_df.loc[:,'PW_PTS_PAINT'] = pw_pts_paints
plyr_box_stats_df.loc[:,'PW_PTS_2PT_MR'] = pw_pts_2pt_mrs
plyr_box_stats_df.loc[:,'PW_PIE_AVG'] = pw_pie_avg
plyr_box_stats_df.loc[:,'PW_USG_PCT_AVG'] = pw_usg_pct_avg

# fill na with 0 on past week stats
plyr_box_stats_df.loc[:,'PW_PFD'] = plyr_box_stats_df['PW_PFD'].fillna(0)
plyr_box_stats_df.loc[:,'PW_PIE_AVG'] = plyr_box_stats_df['PW_PIE_AVG'].fillna(0)
plyr_box_stats_df.loc[:,'PW_USG_PCT_AVG'] = plyr_box_stats_df['PW_USG_PCT_AVG'].fillna(0)

In [512]:
plyr_box_stats_df

,SEASON_YEAR,PLAYER_ID,PLAYER_NAME,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,...,PW_TOV,PW_STL,PW_BLK,PW_POS,PW_PF,PW_PFD,PW_PTS_PAINT,PW_PTS_2PT_MR,PW_PIE_AVG,PW_USG_PCT_AVG
0,2021-22,1629029,Luka Doncic,1610612742,DAL,Dallas Mavericks,0022101219,2022-04-10T00:00:00,DAL vs. SAS,W,...,17,4,1,210,6,8.000000,30,0.140,0.216667,0.431333
1,2021-22,1629029,Luka Doncic,1610612742,DAL,Dallas Mavericks,0022101209,2022-04-08T00:00:00,DAL vs. POR,W,...,17,4,1,222,9,7.666667,28,0.307,0.160333,0.417000
2,2021-22,1629029,Luka Doncic,1610612742,DAL,Dallas Mavericks,0022101190,2022-04-06T00:00:00,DAL @ DET,W,...,14,4,3,229,6,6.333333,38,0.287,0.192000,0.390333
3,2021-22,1629029,Luka Doncic,1610612742,DAL,Dallas Mavericks,0022101167,2022-04-03T00:00:00,DAL @ MIL,W,...,12,2,3,270,8,6.250000,54,0.346,0.220000,0.390000
4,2021-22,1629029,Luka Doncic,1610612742,DAL,Dallas Mavericks,0022101152,2022-04-01T00:00:00,DAL @ WAS,L,...,10,3,2,270,5,6.000000,48,0.346,0.229750,0.358500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60,2021-22,1629029,Luka Doncic,1610612742,DAL,Dallas Mavericks,0022100075,2021-10-29T00:00:00,DAL @ DEN,L,...,16,2,3,220,9,5.333333,40,0.456,0.168667,0.356000
61,2021-22,1629029,Luka Doncic,1610612742,DAL,Dallas Mavericks,0022100069,2021-10-28T00:00:00,DAL vs. SAS,W,...,14,3,2,228,7,6.333333,28,0.678,0.179667,0.329667
62,2021-22,1629029,Luka Doncic,1610612742,DAL,Dallas Mavericks,0022100052,2021-10-26T00:00:00,DAL vs. HOU,W,...,10,1,1,153,4,6.000000,18,0.370,0.154500,0.318000
63,2021-22,1629029,Luka Doncic,1610612742,DAL,Dallas Mavericks,0022100029,2021-10-23T00:00:00,DAL @ TOR,W,...,5,1,0,77,1,6.000000,4,0.222,0.136000,0.276000


In [ ]:
sch_game_dates_obj = [datetime.datetime.strptime(date, '%Y-%m-%d') for date in sch_game_dates]
schedule_df.loc[:,'dateGame_OBJ']  = sch_game_dates_obj

In [513]:
plyr_box_stats_df.columns

Index(['SEASON_YEAR', 'PLAYER_ID', 'PLAYER_NAME', 'TEAM_ID',
       'TEAM_ABBREVIATION', 'TEAM_NAME', 'GAME_ID', 'GAME_DATE', 'MATCHUP',
       'WL', 'MIN', 'FGM', 'FGA', 'FG3M', 'FG3A', 'FTM', 'FTA', 'OREB', 'DREB',
       'AST', 'TOV', 'STL', 'BLK', 'PF', 'PFD', 'PTS', 'USG_PCT', 'PIE',
       'POSS', 'PTS_PAINT', 'PCT_PTS_2PT_MR', 'AGE', 'PLAYER_HEIGHT_INCHES',
       'PLAYER_WEIGHT', 'FGM_PAINT', 'FGM_2PT_MR', 'PW_MIN', 'PW_GAMES',
       'PW_FG3A', 'PW_FG2A', 'PW_OREB', 'PW_DREB', 'PW_ASTS', 'PW_TOV',
       'PW_STL', 'PW_BLK', 'PW_POS', 'PW_PF', 'PW_PFD', 'PW_PTS_PAINT',
       'PW_PTS_2PT_MR', 'PW_PIE_AVG', 'PW_USG_PCT_AVG'],
      dtype='object')

In [514]:
plyr_box_stats_df.to_csv('Luka_Doncic_1629029_2021_22_game_logs.csv')

In [481]:
# to-do: work on injury stats

In [482]:
il_data = pd.read_csv('prosportstransactions_scrape_inactivelist_2010_2023.csv')

In [668]:
il_data

,Date,Team,Acquired,Relinquished,Notes,PLAYER_ID
0,10/26/2010,Blazers,NaN,Elliot Williams,placed on IL,202343.0
1,10/26/2010,Blazers,NaN,Greg Oden,placed on IL with left knee injury (out for se...,201141.0
2,10/26/2010,Blazers,NaN,Joel Przybilla,placed on IL placed on IL recovering from surg...,2038.0
3,10/26/2010,Celtics,NaN,Avery Bradley,placed on IL recovering from surgery on left a...,202340.0
4,10/26/2010,Celtics,NaN,Kendrick Perkins,placed on IL recovering from surgery on right ...,2570.0
...,...,...,...,...,...,...
21439,2/25/2023,Pelicans,Dyson Daniels,NaN,activated from IL,1630700.0
21440,2/25/2023,Pistons,Marvin Bagley III,NaN,activated from IL,1628963.0
21441,2/26/2023,Hawks,John Collins,NaN,activated from IL,1628381.0
21442,2/26/2023,Nuggets,Aaron Gordon,NaN,activated from IL,203932.0


In [672]:
# convert Date string into datetime64 object, add it to il dataframe
il_dates = il_data['Date'].values.tolist()
il_dates_obj = [datetime.datetime.strptime(date, '%m/%d/%Y') for date in il_dates]
il_data.loc[:,'Date_OBJ']  = il_dates_obj

In [673]:
il_data

,Date,Team,Acquired,Relinquished,Notes,PLAYER_ID,Date_OBJ
0,10/26/2010,Blazers,NaN,Elliot Williams,placed on IL,202343.0,2010-10-26
1,10/26/2010,Blazers,NaN,Greg Oden,placed on IL with left knee injury (out for se...,201141.0,2010-10-26
2,10/26/2010,Blazers,NaN,Joel Przybilla,placed on IL placed on IL recovering from surg...,2038.0,2010-10-26
3,10/26/2010,Celtics,NaN,Avery Bradley,placed on IL recovering from surgery on left a...,202340.0,2010-10-26
4,10/26/2010,Celtics,NaN,Kendrick Perkins,placed on IL recovering from surgery on right ...,2570.0,2010-10-26
...,...,...,...,...,...,...,...
21439,2/25/2023,Pelicans,Dyson Daniels,NaN,activated from IL,1630700.0,2023-02-25
21440,2/25/2023,Pistons,Marvin Bagley III,NaN,activated from IL,1628963.0,2023-02-25
21441,2/26/2023,Hawks,John Collins,NaN,activated from IL,1628381.0,2023-02-26
21442,2/26/2023,Nuggets,Aaron Gordon,NaN,activated from IL,203932.0,2023-02-26


In [483]:
mg_data = pd.read_csv('prosportstransactions_scrape_missedgames_2010_2023.csv')

In [674]:
mg_data

,Date,Team,Acquired,Relinquished,Notes,PLAYER_ID
0,8/3/2010,Clippers,NaN,Craig Smith,arthroscopic surgery on right knee (out indefi...,200783.0
1,8/13/2010,Mavericks,NaN,Rodrigue Beaubois,surgery on left foot to repair broken fifth me...,201958.0
2,8/14/2010,Warriors,NaN,Ekpe Udoh,surgery on left wrist (out indefinitely),202327.0
3,9/21/2010,Raptors,NaN,Ed Davis (a),arthroscopic surgery on right kene to repair t...,NaN
4,9/21/2010,Thunder,NaN,Nenad Krstic,surgery on right hand to repair broken finger ...,2420.0
...,...,...,...,...,...,...
14440,2/26/2023,Wizards,NaN,Monte Morris,sore lower back (DTD),1628420.0
14441,2/27/2023,Hornets,NaN,LaMelo Ball,fractured right ankle (out indefinitely),1630163.0
14442,2/27/2023,Pistons,NaN,Bojan Bogdanovic,sore Achilles (DTD),202711.0
14443,2/27/2023,Pistons,NaN,Jaden Ivey,personal reasons (DTD),1631093.0


In [675]:
# convert Date string into datetime64 object, add it to mg dataframe
mg_dates = mg_data['Date'].values.tolist()
mg_dates_obj = [datetime.datetime.strptime(date, '%m/%d/%Y') for date in mg_dates]
mg_data.loc[:,'Date_OBJ']  = mg_dates_obj

In [676]:
mg_data

,Date,Team,Acquired,Relinquished,Notes,PLAYER_ID,Date_OBJ
0,8/3/2010,Clippers,NaN,Craig Smith,arthroscopic surgery on right knee (out indefi...,200783.0,2010-08-03
1,8/13/2010,Mavericks,NaN,Rodrigue Beaubois,surgery on left foot to repair broken fifth me...,201958.0,2010-08-13
2,8/14/2010,Warriors,NaN,Ekpe Udoh,surgery on left wrist (out indefinitely),202327.0,2010-08-14
3,9/21/2010,Raptors,NaN,Ed Davis (a),arthroscopic surgery on right kene to repair t...,NaN,2010-09-21
4,9/21/2010,Thunder,NaN,Nenad Krstic,surgery on right hand to repair broken finger ...,2420.0,2010-09-21
...,...,...,...,...,...,...,...
14440,2/26/2023,Wizards,NaN,Monte Morris,sore lower back (DTD),1628420.0,2023-02-26
14441,2/27/2023,Hornets,NaN,LaMelo Ball,fractured right ankle (out indefinitely),1630163.0,2023-02-27
14442,2/27/2023,Pistons,NaN,Bojan Bogdanovic,sore Achilles (DTD),202711.0,2023-02-27
14443,2/27/2023,Pistons,NaN,Jaden Ivey,personal reasons (DTD),1631093.0,2023-02-27


In [485]:
player_dict = pd.read_csv('df_nba_player_dict.csv')

In [486]:
player_dict

,Unnamed: 0,isActive,isRookie,namePlayer,idPlayer,countSeasons,yearSeasonFirst,yearSeasonLast,idTeam,hasGamesPlayedFlag,urlPlayerStats,urlPlayerThumbnail,urlPlayerHeadshot,urlPlayerActionPhoto,hasHeadShot,hasThumbnail,hasAction,urlPlayerPhoto
0,1,False,False,Alaa Abdelnaby,76001,4,1990,1994,NaN,False,https://stats.nba.com/player/76001,https://stats.nba.com/media/players/230x185/76...,https://ak-static.cms.nba.com/wp-content/uploa...,https://stats.nba.com/media/players/700/76001.png,False,True,True,https://stats.nba.com/media/players/230x185/76...
1,2,False,False,Zaid Abdul-Aziz,76002,9,1968,1977,NaN,False,https://stats.nba.com/player/76002,https://stats.nba.com/media/players/230x185/76...,https://ak-static.cms.nba.com/wp-content/uploa...,https://stats.nba.com/media/players/700/76002.png,False,True,True,https://stats.nba.com/media/players/230x185/76...
2,3,False,False,Kareem Abdul-Jabbar,76003,19,1969,1988,NaN,False,https://stats.nba.com/player/76003,https://stats.nba.com/media/players/230x185/76...,https://ak-static.cms.nba.com/wp-content/uploa...,https://stats.nba.com/media/players/700/76003.png,True,True,True,https://ak-static.cms.nba.com/wp-content/uploa...
3,4,False,False,Mahmoud Abdul-Rauf,51,10,1990,2000,NaN,False,https://stats.nba.com/player/51,https://stats.nba.com/media/players/230x185/51...,https://ak-static.cms.nba.com/wp-content/uploa...,https://stats.nba.com/media/players/700/51.png,False,True,True,https://stats.nba.com/media/players/230x185/51...
4,5,False,False,Tariq Abdul-Wahad,1505,6,1997,2003,NaN,False,https://stats.nba.com/player/1505,https://stats.nba.com/media/players/230x185/15...,https://ak-static.cms.nba.com/wp-content/uploa...,https://stats.nba.com/media/players/700/1505.png,False,True,True,https://stats.nba.com/media/players/230x185/15...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4808,4809,False,False,Ante Zizic,1627790,2,2017,2019,NaN,False,https://stats.nba.com/player/1627790,https://ak-static.cms.nba.com/wp-content/uploa...,https://ak-static.cms.nba.com/wp-content/uploa...,https://stats.nba.com/media/players/700/162779...,True,True,False,https://ak-static.cms.nba.com/wp-content/uploa...
4809,4810,False,False,Jim Zoet,78647,0,1982,1982,NaN,False,https://stats.nba.com/player/78647,https://stats.nba.com/media/players/230x185/78...,https://ak-static.cms.nba.com/wp-content/uploa...,https://stats.nba.com/media/players/700/78647.png,False,True,True,https://stats.nba.com/media/players/230x185/78...
4810,4811,False,False,Bill Zopf,78648,0,1970,1970,NaN,False,https://stats.nba.com/player/78648,https://stats.nba.com/media/players/230x185/78...,https://ak-static.cms.nba.com/wp-content/uploa...,https://stats.nba.com/media/players/700/78648.png,False,True,True,https://stats.nba.com/media/players/230x185/78...
4811,4812,True,False,Ivica Zubac,1627826,6,2016,2022,NaN,False,https://stats.nba.com/player/1627826,https://stats.nba.com/media/players/230x185/16...,https://ak-static.cms.nba.com/wp-content/uploa...,https://stats.nba.com/media/players/700/162782...,True,True,True,https://ak-static.cms.nba.com/wp-content/uploa...


In [487]:
mg_id = []

num_cannot_find = 0

for index, row in mg_data.iterrows():
    namePlayer = row['Acquired']
    if pd.isnull(row['Acquired']):
        namePlayer = row['Relinquished']
    player = player_dict.loc[player_dict['namePlayer'] == namePlayer]
    if (player.empty):
        mg_id.append(np.nan)
        print('cannot find: ' + str(namePlayer))
        num_cannot_find = num_cannot_find + 1
    else:
        mg_id.append(round(player['idPlayer'].values[0],0))
        
print('cannot find: ' + str(num_cannot_find))

cannot find: Ed Davis (a)
cannot find: Jeff Pendergraph / Jeff Ayres
cannot find: Louis Amundson / Lou Amundson
cannot find: Luc Richard Mbah a Moute / Luc Mbah a Moute
cannot find: Nene / Nene Hilario / Maybyner Hilario
cannot find: Nene / Nene Hilario / Maybyner Hilario
cannot find: Nene / Nene Hilario / Maybyner Hilario
cannot find: Mike Dunleavy Jr.
cannot find: Mike Dunleavy Jr.
cannot find: John Wall (a)
cannot find: Marcus Thornton (T.)
cannot find: John Wall (a)
cannot find: Predrag Stojakovic / Peja Stojakovic
cannot find: John Wall (a)
cannot find: Predrag Stojakovic / Peja Stojakovic
cannot find: Predrag Stojakovic / Peja Stojakovic
cannot find: J.J. Redick
cannot find: J.J. Redick
cannot find: Predrag Stojakovic / Peja Stojakovic
cannot find: D.J. White
cannot find: Predrag Stojakovic / Peja Stojakovic
cannot find: John Wall (a)
cannot find: Zaur Pachulia / Zaza Pachulia
cannot find: D.J. White
cannot find: Predrag Stojakovic / Peja Stojakovic
cannot find: Predrag Stojakovi

cannot find: Larry Sanders (b)
cannot find: C.J. Miles
cannot find: D.J. Augustine
cannot find: Marcus Thornton (T.)
cannot find: C.J. Miles
cannot find: D.J. Augustine
cannot find: Malcolm Lee / Malcom Lee
cannot find: Marcus Thornton (T.)
cannot find: Louis Williams / Lou Williams
cannot find: Emanuel Ginobili / Manu Ginobili
cannot find: Louis Williams / Lou Williams
cannot find: J.J. Redick
cannot find: Emanuel Ginobili / Manu Ginobili
cannot find: Amare Stoudemire / Amar'e Stoudemire
cannot find: Luc Richard Mbah a Moute / Luc Mbah a Moute
cannot find: James Jones (b)
cannot find: John Wall (a)
cannot find: Chris Smith (c)
cannot find: Amare Stoudemire / Amar'e Stoudemire
cannot find: Amare Stoudemire / Amar'e Stoudemire
cannot find: Emanuel Ginobili / Manu Ginobili
cannot find: Maurice Harkless / Moe Harkless
cannot find: Gerald Henderson (b)
cannot find: Maurice Harkless / Moe Harkless
cannot find: Jose Juan Barea / Jose Barea / J.J. Barea
cannot find: Maurice Williams / Mo Will

cannot find: Zaur Pachulia / Zaza Pachulia
cannot find: Ed Davis (a)
cannot find: Greg Smith (a)
cannot find: Zaur Pachulia / Zaza Pachulia
cannot find: Mike James (Lamont)
cannot find: Greg Smith (a)
cannot find: Nene / Nene Hilario / Maybyner Hilario
cannot find: Zaur Pachulia / Zaza Pachulia
cannot find: Nene / Nene Hilario / Maybyner Hilario
cannot find: Mike Conley Jr.
cannot find: Zaur Pachulia / Zaza Pachulia
cannot find: Mike Conley Jr.
cannot find: Emanuel Ginobili / Manu Ginobili
cannot find: Zaur Pachulia / Zaza Pachulia
cannot find: Jeffery Taylor / Jeff Taylor (b)
cannot find: Zaur Pachulia / Zaza Pachulia
cannot find: Metta World Peace / Ron Artest
cannot find: Metta World Peace / Ron Artest
cannot find: Greg Smith (a)
cannot find: Metta World Peace / Ron Artest
cannot find: Greg Smith (a)
cannot find: Greg Smith (a)
cannot find: Jeffery Taylor / Jeff Taylor (b)
cannot find: Jeffery Taylor / Jeff Taylor (b)
cannot find: Metta World Peace / Ron Artest
cannot find: Greg Smi

cannot find: Mike Conley Jr.
cannot find: J.J. Redick
cannot find: Maurice Williams / Mo Williams
cannot find: Maurice Williams / Mo Williams
cannot find: Zaur Pachulia / Zaza Pachulia
cannot find: J.J. Redick
cannot find: Mike Conley Jr.
cannot find: Maurice Williams / Mo Williams
cannot find: Maurice Williams / Mo Williams
cannot find: J.J. Redick
cannot find: Hidayet Turkoglu / Hedo Turkoglu
cannot find: Hidayet Turkoglu / Hedo Turkoglu
cannot find: Zaur Pachulia / Zaza Pachulia
cannot find: J.J. Redick
cannot find: James Ennis
cannot find: Larry Sanders (b)
cannot find: James Ennis
cannot find: Larry Sanders (b)
cannot find: Luc Richard Mbah a Moute / Luc Mbah a Moute
cannot find: Luc Richard Mbah a Moute / Luc Mbah a Moute
cannot find: Louis Williams / Lou Williams
cannot find: Enes Kanter
cannot find: Enes Kanter
cannot find: Wesley Matthews / Wes Matthews Jr.
cannot find: Louis Amundson / Lou Amundson
cannot find: Louis Amundson / Lou Amundson
cannot find: Mike Conley Jr.
cannot

cannot find: DeAndre' Bembry / DeAndre Bembry
cannot find: DeAndre' Bembry / DeAndre Bembry
cannot find: Maurice Harkless / Moe Harkless
cannot find: Patrick Mills / Patty Mills
cannot find: Maurice Harkless / Moe Harkless
cannot find: Wesley Johnson / Wes Johnson
cannot find: DeAndre' Bembry / DeAndre Bembry
cannot find: Ed Davis (a)
cannot find: Patrick Mills / Patty Mills
cannot find: DeAndre' Bembry / DeAndre Bembry
cannot find: Wesley Johnson / Wes Johnson
cannot find: (William) Tony Parker
cannot find: Ed Davis (a)
cannot find: Jose Juan Barea / Jose Barea / J.J. Barea
cannot find: Ed Davis (a)
cannot find: Jose Juan Barea / Jose Barea / J.J. Barea
cannot find: Emanuel Ginobili / Manu Ginobili
cannot find: Gerald Green (b)
cannot find: Wesley Matthews / Wes Matthews Jr.
cannot find: D.J. Augustine
cannot find: Emanuel Ginobili / Manu Ginobili
cannot find: Ed Davis (a)
cannot find: Maurice Harkless / Moe Harkless
cannot find: Enes Kanter
cannot find: D.J. Augustine
cannot find: En

cannot find: Marcus Morris
cannot find: Marcus Morris
cannot find: Wesley Matthews / Wes Matthews Jr.
cannot find: Jose Juan Barea / Jose Barea / J.J. Barea
cannot find: J.R. Smith
cannot find: Guillermo Hernangomez / Willy Hernangomez
cannot find: Jose Juan Barea / Jose Barea / J.J. Barea
cannot find: Guillermo Hernangomez / Willy Hernangomez
cannot find: (William) Tony Parker
cannot find: (William) Tony Parker
cannot find: Gerald Green (b)
cannot find: Zhou Qi / Qi Zhou
cannot find: Maximilian Kleber / Maxi Kleber
cannot find: Gerald Green (b)
cannot find: Nene / Nene Hilario / Maybyner Hilario
cannot find: Wayne Selden Jr.
cannot find: John Wall (a)
cannot find: Wayne Selden Jr.
cannot find: John Wall (a)
cannot find: John Wall (a)
cannot find: Louis Williams / Lou Williams
cannot find: John Wall (a)
cannot find: Zhou Qi / Qi Zhou
cannot find: John Wall (a)
cannot find: John Wall (a)
cannot find: Wesley Matthews / Wes Matthews Jr.
cannot find: John Wall (a)
cannot find: D.J. Augusti

cannot find: Marcus Morris
cannot find: Nah'Shon Hyland / Bones Hyland
cannot find: Nah'Shon Hyland / Bones Hyland
cannot find: Marcus Morris
cannot find: Nicolas Claxton / Nic Claxton
cannot find: Nicolas Claxton / Nic Claxton
cannot find: Marcus Morris
cannot find: Maximilian Kleber / Maxi Kleber
cannot find: Nicolas Claxton / Nic Claxton
cannot find: Maximilian Kleber / Maxi Kleber
cannot find: Nicolas Claxton / Nic Claxton
cannot find: Wesley Matthews / Wes Matthews Jr.
cannot find: Wesley Matthews / Wes Matthews Jr.
cannot find: Nah'Shon Hyland / Bones Hyland
cannot find: Nah'Shon Hyland / Bones Hyland
cannot find: (Sean) Chris Smith
cannot find: Cameron Thomas / Cam Thomas
cannot find: Cameron Thomas / Cam Thomas
cannot find: Louis Williams / Lou Williams
cannot find: (Sean) Chris Smith
cannot find: Louis Williams / Lou Williams
cannot find: Marcus Morris
cannot find: Marcus Morris
cannot find: Maximilian Kleber / Maxi Kleber
cannot find: Ja'Vonte Smart / Javonte Smart
cannot fin

In [489]:
len(mg_id)

14445

In [490]:
mg_data.loc[:,'PLAYER_ID'] = mg_id

In [533]:
il_id = []

num_cannot_find = 0

for index, row in il_data.iterrows():
    namePlayer = row['Acquired']
    if pd.isnull(row['Acquired']):
        namePlayer = row['Relinquished']
    player = player_dict.loc[player_dict['namePlayer'] == namePlayer]
    if (player.empty):
        il_id.append(np.nan)
        print('cannot find: ' + str(namePlayer))
        num_cannot_find = num_cannot_find + 1
    else:
        il_id.append(int(round(player['idPlayer'].values[0],0)))
        
print('cannot find: ' + str(num_cannot_find))

cannot find: Matt Janning
cannot find: Maurice Williams / Mo Williams
cannot find: Alexis Ajinca / Alex Ajinca
cannot find: A.J. Price
cannot find: Ed Davis (a)
cannot find: Byron Mullens / B.J. Mullens
cannot find: Louis Amundson / Lou Amundson
cannot find: Eugene Jeter / Pooh Jeter
cannot find: Ishmael Smith / Ish Smith
cannot find: Patrick Mills / Patty Mills
cannot find: Maurice Williams / Mo Williams
cannot find: Predrag Stojakovic / Peja Stojakovic
cannot find: Joe Smith (Leynard)
cannot find: Patrick Mills / Patty Mills
cannot find: Eugene Jeter / Pooh Jeter
cannot find: Ishmael Smith / Ish Smith
cannot find: Eugene Jeter / Pooh Jeter
cannot find: Joe Smith (Leynard)
cannot find: Ed Davis (a)
cannot find: Byron Mullens / B.J. Mullens
cannot find: Maurice Williams / Mo Williams
cannot find: Marcus Thornton (T.)
cannot find: Predrag Stojakovic / Peja Stojakovic
cannot find: Joe Smith (Leynard)
cannot find: Ed Davis (a)
cannot find: Byron Mullens / B.J. Mullens
cannot find: Gerald 

cannot find: Emanuel Ginobili / Manu Ginobili
cannot find: Emanuel Ginobili / Manu Ginobili
cannot find: Ishmael Smith / Ish Smith
cannot find: Ishmael Smith / Ish Smith
cannot find: Patrick Mills / Patty Mills
cannot find: Emanuel Ginobili / Manu Ginobili
cannot find: Chris Wright (b)
cannot find: Maurice Williams / Mo Williams
cannot find: Amare Stoudemire / Amar'e Stoudemire
cannot find: Greg Smith (a)
cannot find: Patrick Mills / Patty Mills
cannot find: Malcolm Thomas (b)
cannot find: Bill Walker
cannot find: Luc Richard Mbah a Moute / Luc Mbah a Moute
cannot find: Luc Richard Mbah a Moute / Luc Mbah a Moute
cannot find: Hidayet Turkoglu / Hedo Turkoglu
cannot find: Chris Wright (b)
cannot find: Maurice Williams / Mo Williams
cannot find: Chris Johnson (a)
cannot find: Zaur Pachulia / Zaza Pachulia
cannot find: Chris Johnson (a)
cannot find: Amare Stoudemire / Amar'e Stoudemire
cannot find: Luc Richard Mbah a Moute / Luc Mbah a Moute
cannot find: C.J. Miles
cannot find: (William) 

cannot find: (William) Tony Parker
cannot find: Nene / Nene Hilario / Maybyner Hilario
cannot find: Malcolm Thomas (b)
cannot find: Amare Stoudemire / Amar'e Stoudemire
cannot find: (William) Tony Parker
cannot find: Malcolm Thomas (b)
cannot find: Tony Mitchell (b)
cannot find: Zaur Pachulia / Zaza Pachulia
cannot find: Larry Sanders (b)
cannot find: Ognjen Kuzmic / Ognen Kuzmic
cannot find: Ognjen Kuzmic / Ognen Kuzmic
cannot find: James Jones (b)
cannot find: James Jones (b)
cannot find: Metta World Peace / Ron Artest
cannot find: Luc Richard Mbah a Moute / Luc Mbah a Moute
cannot find: J.J. Redick
cannot find: Luc Richard Mbah a Moute / Luc Mbah a Moute
cannot find: Jeffery Taylor / Jeff Taylor (b)
cannot find: A.J. Price
cannot find: Byron Mullens / B.J. Mullens
cannot find: A.J. Price
cannot find: Byron Mullens / B.J. Mullens
cannot find: Tony Mitchell (b)
cannot find: A.J. Price
cannot find: Ray McCallum Jr.
cannot find: Malcolm Thomas (b)
cannot find: Ray McCallum Jr.
cannot fi

cannot find: James Michael McAdoo / James McAdoo (Michael)
cannot find: Emanuel Ginobili / Manu Ginobili
cannot find: James Michael McAdoo / James McAdoo (Michael)
cannot find: Marcus Thornton (T.)
cannot find: Greg Smith (a)
cannot find: Greg Smith (a)
cannot find: Jeff Ayres / Jeff Pendergraph
cannot find: Marcus Thornton (T.)
cannot find: Greg Smith (a)
cannot find: Jose Juan Barea / Jose Barea / J.J. Barea
cannot find: Jeff Ayres / Jeff Pendergraph
cannot find: James Michael McAdoo / James McAdoo (Michael)
cannot find: Nene / Nene Hilario / Maybyner Hilario
cannot find: James Michael McAdoo / James McAdoo (Michael)
cannot find: James Michael McAdoo / James McAdoo (Michael)
cannot find: John Wall (a)
cannot find: James Michael McAdoo / James McAdoo (Michael)
cannot find: Mike Conley Jr.
cannot find: Greg Smith (a)
cannot find: Nene / Nene Hilario / Maybyner Hilario
cannot find: John Wall (a)
cannot find: Nene / Nene Hilario / Maybyner Hilario
cannot find: James Michael McAdoo / Jame

cannot find: Guillermo Hernangomez / Willy Hernangomez
cannot find: Marcelinho Huertas / Marcelo Huertas
cannot find: A.J. Hammons
cannot find: (William) Tony Parker
cannot find: Kahlil Felder / Kay Felder
cannot find: Georgios Papagiannis / Giorgios Papagiannis / George Papagiannis
cannot find: Stephen Zimmerman Jr.
cannot find: John Wall (a)
cannot find: Danuel House
cannot find: Georgios Papagiannis / Giorgios Papagiannis / George Papagiannis
cannot find: Maurice Ndour / Maurice N'dour
cannot find: Marcelinho Huertas / Marcelo Huertas
cannot find: Danuel House
cannot find: John Wall (a)
cannot find: Metta World Peace / Ron Artest
cannot find: Maurice Ndour / Maurice N'dour
cannot find: Stephen Zimmerman Jr.
cannot find: Metta World Peace / Ron Artest
cannot find: James Michael McAdoo / James McAdoo (Michael)
cannot find: Maurice Ndour / Maurice N'dour
cannot find: Stephen Zimmerman Jr.
cannot find: (William) Tony Parker
cannot find: Danuel House
cannot find: Wesley Johnson / Wes Joh

cannot find: Wesley Johnson / Wes Johnson
cannot find: Mike Conley Jr.
cannot find: Mike Conley Jr.
cannot find: Raulzinho Neto / Raul Neto
cannot find: Raulzinho Neto / Raul Neto
cannot find: Stephen Zimmerman Jr.
cannot find: Sheldon McClellan / Sheldon Mac
cannot find: Raulzinho Neto / Raul Neto
cannot find: Kahlil Felder / Kay Felder
cannot find: DeAndre' Bembry / DeAndre Bembry
cannot find: Kahlil Felder / Kay Felder
cannot find: James Ennis
cannot find: DeAndre' Bembry / DeAndre Bembry
cannot find: Stephen Zimmerman Jr.
cannot find: Jose Juan Barea / Jose Barea / J.J. Barea
cannot find: Sheldon McClellan / Sheldon Mac
cannot find: John Wall (a)
cannot find: Kahlil Felder / Kay Felder
cannot find: Walter Tavares / Edy Tavares
cannot find: Raulzinho Neto / Raul Neto
cannot find: DeAndre' Bembry / DeAndre Bembry
cannot find: Raulzinho Neto / Raul Neto
cannot find: Norris Cole (a)
cannot find: Norris Cole (a)
cannot find: James Ennis
cannot find: (William) Tony Parker
cannot find: Ne

cannot find: Wesley Matthews / Wes Matthews Jr.
cannot find: Emanuel Ginobili / Manu Ginobili
cannot find: Jose Juan Barea / Jose Barea / J.J. Barea
cannot find: Jose Juan Barea / Jose Barea / J.J. Barea
cannot find: C.J. Miles
cannot find: Marcus Morris
cannot find: C.J. Miles
cannot find: R.J. Hunter
cannot find: Zhou Qi / Qi Zhou
cannot find: Marcus Morris
cannot find: Juancho Hernangomez / Juan Hernangomez
cannot find: Danuel House
cannot find: Luc Richard Mbah a Moute / Luc Mbah a Moute
cannot find: Maurice Harkless / Moe Harkless
cannot find: Georgios Papagiannis / Giorgios Papagiannis / George Papagiannis
cannot find: Nene / Nene Hilario / Maybyner Hilario
cannot find: R.J. Hunter
cannot find: Zhou Qi / Qi Zhou
cannot find: Luc Richard Mbah a Moute / Luc Mbah a Moute
cannot find: Marcus Morris
cannot find: Enes Kanter
cannot find: John Wall (a)
cannot find: Ed Davis (a)
cannot find: Georgios Papagiannis / Giorgios Papagiannis / George Papagiannis
cannot find: DeAndre' Bembry / D

cannot find: Wesley Matthews / Wes Matthews Jr.
cannot find: Cameron Reddish / Cam Reddish
cannot find: Cameron Reddish / Cam Reddish
cannot find: Nicolas Claxton / Nic Claxton
cannot find: Marcus Morris
cannot find: C.J. Miles
cannot find: D.J. Augustine
cannot find: Marcus Morris
cannot find: Maurice Harkless / Moe Harkless
cannot find: Mike Conley Jr.
cannot find: Maurice Harkless / Moe Harkless
cannot find: Nicolas Claxton / Nic Claxton
cannot find: Enes Kanter / Enes Freedom
cannot find: Nicolas Claxton / Nic Claxton
cannot find: Mike Conley Jr.
cannot find: Mike Conley Jr.
cannot find: Moritz Wagner / Moe Wagner
cannot find: Enes Kanter / Enes Freedom
cannot find: Cameron Reddish / Cam Reddish
cannot find: Jose Juan Barea / Jose Barea / J.J. Barea
cannot find: Nicolas Claxton / Nic Claxton
cannot find: Nene / Nene Hilario / Maybyner Hilario
cannot find: Ed Davis (Adam)
cannot find: Jose Juan Barea / Jose Barea / J.J. Barea
cannot find: Nicolas Claxton / Nic Claxton
cannot find: M

cannot find: Ishmael Smith / Ish Smith
cannot find: Maurice Harkless / Moe Harkless
cannot find: Nah'Shon Hyland / Bones Hyland
cannot find: Cameron Reddish / Cam Reddish
cannot find: David Johnson (Ricardo)
cannot find: Wesley Matthews / Wes Matthews Jr.
cannot find: Ishmael Smith / Ish Smith
cannot find: Ignas Brazdeikis / Iggy Brazdeikis
cannot find: Juancho Hernangomez / Juan Hernangomez
cannot find: Mohamed Bamba / Mo Bamba
cannot find: Moritz Wagner / Moe Wagner
cannot find: Ruben Nembhard / R.J. Nembhard Jr.
cannot find: Marcus Morris
cannot find: Nicolas Claxton / Nic Claxton
cannot find: Ruben Nembhard / R.J. Nembhard Jr.
cannot find: J.T. Thor
cannot find: Maximilian Kleber / Maxi Kleber
cannot find: Wesley Matthews / Wes Matthews Jr.
cannot find: Ed Davis (Adam)
cannot find: Daulton Hommes
cannot find: Louis Williams / Lou Williams
cannot find: Enes Freedom / Enes Kanter
cannot find: Cameron Thomas / Cam Thomas
cannot find: J.T. Thor
cannot find: C.J. Miles
cannot find: Just

In [534]:
il_data.loc[:,'PLAYER_ID'] = il_id

In [494]:
len(il_id)


21444

In [495]:
il_data.loc[:,'PLAYER_ID'] = il_id


In [536]:
il_data

,Date,Team,Acquired,Relinquished,Notes,PLAYER_ID
0,10/26/2010,Blazers,NaN,Elliot Williams,placed on IL,202343.0
1,10/26/2010,Blazers,NaN,Greg Oden,placed on IL with left knee injury (out for se...,201141.0
2,10/26/2010,Blazers,NaN,Joel Przybilla,placed on IL placed on IL recovering from surg...,2038.0
3,10/26/2010,Celtics,NaN,Avery Bradley,placed on IL recovering from surgery on left a...,202340.0
4,10/26/2010,Celtics,NaN,Kendrick Perkins,placed on IL recovering from surgery on right ...,2570.0
...,...,...,...,...,...,...
21439,2/25/2023,Pelicans,Dyson Daniels,NaN,activated from IL,1630700.0
21440,2/25/2023,Pistons,Marvin Bagley III,NaN,activated from IL,1628963.0
21441,2/26/2023,Hawks,John Collins,NaN,activated from IL,1628381.0
21442,2/26/2023,Nuggets,Aaron Gordon,NaN,activated from IL,203932.0


In [497]:

#player = player_dict.loc[player_dict['namePlayer'] == 'LeBron James']
player = player_dict.loc[player_dict['namePlayer'] == 'Luka Doncic']
player['idPlayer'].values[0]

1629029

In [678]:
player_il = il_data.loc[il_data['PLAYER_ID'] == 1629029]
player_mg = mg_data.loc[mg_data['PLAYER_ID'] == 1629029]


In [679]:
player_il

,Date,Team,Acquired,Relinquished,Notes,PLAYER_ID,Date_OBJ
12875,12/2/2018,Mavericks,NaN,Luka Doncic,placed on IL with strained right hip,1629029.0,2018-12-02
12885,12/4/2018,Mavericks,Luka Doncic,NaN,activated from IL,1629029.0,2018-12-04
13607,2/22/2019,Mavericks,NaN,Luka Doncic,placed on IL with sore right ankle,1629029.0,2019-02-22
13640,2/25/2019,Mavericks,Luka Doncic,NaN,activated from IL,1629029.0,2019-02-25
13813,3/16/2019,Mavericks,NaN,Luka Doncic,placed on IL with bruised right knee,1629029.0,2019-03-16
13839,3/18/2019,Mavericks,Luka Doncic,NaN,activated from IL,1629029.0,2019-03-18
13955,3/31/2019,Mavericks,NaN,Luka Doncic,placed on IL with bruised right thigh,1629029.0,2019-03-31
13987,4/3/2019,Mavericks,Luka Doncic,NaN,activated from IL,1629029.0,2019-04-03
14004,4/5/2019,Mavericks,NaN,Luka Doncic,placed on IL with bruised right thigh,1629029.0,2019-04-05
14043,4/9/2019,Mavericks,Luka Doncic,NaN,activated from IL,1629029.0,2019-04-09


In [680]:
player_mg

,Date,Team,Acquired,Relinquished,Notes,PLAYER_ID,Date_OBJ
11133,1/31/2019,Mavericks,NaN,Luka Doncic,sore left ankle (DTD),1629029.0,2019-01-31
11148,2/2/2019,Mavericks,Luka Doncic,NaN,returned to lineup,1629029.0,2019-02-02
11409,3/30/2019,Mavericks,NaN,Luka Doncic,bruised right thigh (DTD),1629029.0,2019-03-30
11492,4/10/2019,Mavericks,NaN,Luka Doncic,left thigh injury (out for season),1629029.0,2019-04-10
11831,12/15/2019,Mavericks,NaN,Luka Doncic,sprained right ankle (DTD),1629029.0,2019-12-15
12266,8/9/2020,Mavericks,NaN,Luka Doncic,right ankle injury (DTD),1629029.0,2020-08-09
12276,8/11/2020,Mavericks,Luka Doncic,NaN,returned to lineup,1629029.0,2020-08-11
12405,1/3/2021,Mavericks,NaN,Luka Doncic,bruised left quadricep (DTD),1629029.0,2021-01-03
12412,1/4/2021,Mavericks,Luka Doncic,NaN,returned to lineup,1629029.0,2021-01-04
12799,3/11/2021,Mavericks,NaN,Luka Doncic,right ankle injury (DTD),1629029.0,2021-03-11


In [661]:
len(plyr_box_stats_df)

65

In [703]:
first_game_season = plyr_box_stats_df.iloc[len(plyr_box_stats_df) - 1]
first_game_season_date = first_game_season['GAME_DATE_OBJ']
first_game_season_date

Timestamp('2021-10-21 00:00:00')

In [689]:
player_mg_before_season = player_mg.loc[player_mg['Date_OBJ'] < first_game_season_date]
player_mg_before_season = player_mg_before_season.loc[pd.isnull(player_mg_before_season['Acquired'])]
player_mg_before_season

,Date,Team,Acquired,Relinquished,Notes,PLAYER_ID,Date_OBJ
11133,1/31/2019,Mavericks,NaN,Luka Doncic,sore left ankle (DTD),1629029.0,2019-01-31
11409,3/30/2019,Mavericks,NaN,Luka Doncic,bruised right thigh (DTD),1629029.0,2019-03-30
11492,4/10/2019,Mavericks,NaN,Luka Doncic,left thigh injury (out for season),1629029.0,2019-04-10
11831,12/15/2019,Mavericks,NaN,Luka Doncic,sprained right ankle (DTD),1629029.0,2019-12-15
12266,8/9/2020,Mavericks,NaN,Luka Doncic,right ankle injury (DTD),1629029.0,2020-08-09
12405,1/3/2021,Mavericks,NaN,Luka Doncic,bruised left quadricep (DTD),1629029.0,2021-01-03
12799,3/11/2021,Mavericks,NaN,Luka Doncic,right ankle injury (DTD),1629029.0,2021-03-11


In [704]:
last_mg = player_mg_before_season.iloc[len(player_mg_before_season) - 1]
last_mg_date = last_mg['Date_OBJ']
last_mg_date

Timestamp('2021-03-11 00:00:00')

In [698]:
player_il_before_season = player_il.loc[player_il['Date_OBJ'] < first_game_season_date]
player_il_before_season = player_il_before_season.loc[pd.isnull(player_il_before_season['Acquired'])]
player_il_before_season

,Date,Team,Acquired,Relinquished,Notes,PLAYER_ID,Date_OBJ
12875,12/2/2018,Mavericks,NaN,Luka Doncic,placed on IL with strained right hip,1629029.0,2018-12-02
13607,2/22/2019,Mavericks,NaN,Luka Doncic,placed on IL with sore right ankle,1629029.0,2019-02-22
13813,3/16/2019,Mavericks,NaN,Luka Doncic,placed on IL with bruised right knee,1629029.0,2019-03-16
13955,3/31/2019,Mavericks,NaN,Luka Doncic,placed on IL with bruised right thigh,1629029.0,2019-03-31
14004,4/5/2019,Mavericks,NaN,Luka Doncic,placed on IL with bruised right thigh,1629029.0,2019-04-05
14631,12/16/2019,Mavericks,NaN,Luka Doncic,placed on IL with sprained right ankle,1629029.0,2019-12-16
15104,1/31/2020,Mavericks,NaN,Luka Doncic,placed on IL with sprained right ankle,1629029.0,2020-01-31
15289,2/22/2020,Mavericks,NaN,Luka Doncic,placed on IL with sore right ankle,1629029.0,2020-02-22
15356,3/1/2020,Mavericks,NaN,Luka Doncic,placed on IL with sprained left thumb,1629029.0,2020-03-01
16461,3/3/2021,Mavericks,NaN,Luka Doncic,placed on IL with tightness in lower back,1629029.0,2021-03-03


In [701]:
len(player_il_before_season) - 1

12

In [702]:
last_il = player_il_before_season.iloc[len(player_il_before_season) - 1]
last_il_date = last_il['Date_OBJ']
last_il_date

Timestamp('2021-04-29 00:00:00')

In [706]:
last_injury_date = max(last_il_date, last_mg_date)
last_injury_date

Timestamp('2021-04-29 00:00:00')

In [707]:
num_career_injuries = len(player_mg_before_season) + len(player_il_before_season)
num_career_injuries

20

In [575]:
schedule_df = pd.read_csv("schedule_2021_22.csv")

In [576]:
# convert dateGame string into datetime64 object, add it to schedule dataframe
sch_game_dates = schedule_df['dateGame'].values.tolist()
sch_game_dates_obj = [datetime.datetime.strptime(date, '%Y-%m-%d') for date in sch_game_dates]
schedule_df.loc[:,'dateGame_OBJ']  = sch_game_dates_obj

In [597]:
schedule_df

,Unnamed: 0,typeSeason,slugSeason,idGame,dateGame,slugMatchup,slugTeamWinner,slugTeamLoser,numberGameDay,dateGame_OBJ
0,1,Regular Season,2021-22,22100001,2021-10-19,MIL vs. BKN,MIL,BKN,1,2021-10-19
1,2,Regular Season,2021-22,22100002,2021-10-19,LAL vs. GSW,GSW,LAL,2,2021-10-19
2,3,Regular Season,2021-22,22100003,2021-10-20,CHA vs. IND,CHA,IND,1,2021-10-20
3,4,Regular Season,2021-22,22100004,2021-10-20,DET vs. CHI,CHI,DET,2,2021-10-20
4,5,Regular Season,2021-22,22100005,2021-10-20,NYK vs. BOS,NYK,BOS,3,2021-10-20
...,...,...,...,...,...,...,...,...,...,...
1225,1226,Regular Season,2021-22,22101226,2022-04-10,TOR @ NYK,NYK,TOR,11,2022-04-10
1226,1227,Regular Season,2021-22,22101227,2022-04-10,MIA @ ORL,ORL,MIA,12,2022-04-10
1227,1228,Regular Season,2021-22,22101228,2022-04-10,DET @ PHI,PHI,DET,13,2022-04-10
1228,1229,Regular Season,2021-22,22101229,2022-04-10,PHX vs. SAC,SAC,PHX,14,2022-04-10


In [578]:
# make a dictionary of team to list of games played 
team_dates_dict = {}
for team in team_abbrev:
    team_games = schedule_df.loc[(schedule_df['slugTeamWinner'] == team) | (schedule_df['slugTeamLoser'] == team)]
    team_dates_dict[team]=team_games

In [579]:
team_dates_dict

{'BOS':       Unnamed: 0      typeSeason slugSeason    idGame    dateGame  \
 4              5  Regular Season    2021-22  22100005  2021-10-20   
 19            20  Regular Season    2021-22  22100020  2021-10-22   
 36            37  Regular Season    2021-22  22100037  2021-10-24   
 40            41  Regular Season    2021-22  22100041  2021-10-25   
 55            56  Regular Season    2021-22  22100056  2021-10-27   
 ...          ...             ...        ...       ...         ...   
 1152        1153  Regular Season    2021-22  22101153  2022-04-01   
 1165        1166  Regular Season    2021-22  22101166  2022-04-03   
 1192        1193  Regular Season    2021-22  22101193  2022-04-06   
 1197        1198  Regular Season    2021-22  22101198  2022-04-07   
 1222        1223  Regular Season    2021-22  22101223  2022-04-10   
 
       slugMatchup slugTeamWinner slugTeamLoser  numberGameDay dateGame_OBJ  
 4     NYK vs. BOS            NYK           BOS              3   2021-10-

In [551]:
plyr_box_stats_df

,SEASON_YEAR,PLAYER_ID,PLAYER_NAME,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,...,PW_STL,PW_BLK,PW_POS,PW_PF,PW_PFD,PW_PTS_PAINT,PW_PTS_2PT_MR,PW_PIE_AVG,PW_USG_PCT_AVG,GAME_DATE_OBJ
0,2021-22,1629029,Luka Doncic,1610612742,DAL,Dallas Mavericks,0022101219,2022-04-10T00:00:00,DAL vs. SAS,W,...,4,1,210,6,8.000000,30,0.140,0.216667,0.431333,2022-04-10
1,2021-22,1629029,Luka Doncic,1610612742,DAL,Dallas Mavericks,0022101209,2022-04-08T00:00:00,DAL vs. POR,W,...,4,1,222,9,7.666667,28,0.307,0.160333,0.417000,2022-04-08
2,2021-22,1629029,Luka Doncic,1610612742,DAL,Dallas Mavericks,0022101190,2022-04-06T00:00:00,DAL @ DET,W,...,4,3,229,6,6.333333,38,0.287,0.192000,0.390333,2022-04-06
3,2021-22,1629029,Luka Doncic,1610612742,DAL,Dallas Mavericks,0022101167,2022-04-03T00:00:00,DAL @ MIL,W,...,2,3,270,8,6.250000,54,0.346,0.220000,0.390000,2022-04-03
4,2021-22,1629029,Luka Doncic,1610612742,DAL,Dallas Mavericks,0022101152,2022-04-01T00:00:00,DAL @ WAS,L,...,3,2,270,5,6.000000,48,0.346,0.229750,0.358500,2022-04-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60,2021-22,1629029,Luka Doncic,1610612742,DAL,Dallas Mavericks,0022100075,2021-10-29T00:00:00,DAL @ DEN,L,...,2,3,220,9,5.333333,40,0.456,0.168667,0.356000,2021-10-29
61,2021-22,1629029,Luka Doncic,1610612742,DAL,Dallas Mavericks,0022100069,2021-10-28T00:00:00,DAL vs. SAS,W,...,3,2,228,7,6.333333,28,0.678,0.179667,0.329667,2021-10-28
62,2021-22,1629029,Luka Doncic,1610612742,DAL,Dallas Mavericks,0022100052,2021-10-26T00:00:00,DAL vs. HOU,W,...,1,1,153,4,6.000000,18,0.370,0.154500,0.318000,2021-10-26
63,2021-22,1629029,Luka Doncic,1610612742,DAL,Dallas Mavericks,0022100029,2021-10-23T00:00:00,DAL @ TOR,W,...,1,0,77,1,6.000000,4,0.222,0.136000,0.276000,2021-10-23


In [552]:
plyr_box_stats_df.columns

Index(['SEASON_YEAR', 'PLAYER_ID', 'PLAYER_NAME', 'TEAM_ID',
       'TEAM_ABBREVIATION', 'TEAM_NAME', 'GAME_ID', 'GAME_DATE', 'MATCHUP',
       'WL', 'MIN', 'FGM', 'FGA', 'FG3M', 'FG3A', 'FTM', 'FTA', 'OREB', 'DREB',
       'AST', 'TOV', 'STL', 'BLK', 'PF', 'PFD', 'PTS', 'USG_PCT', 'PIE',
       'POSS', 'PTS_PAINT', 'PCT_PTS_2PT_MR', 'AGE', 'PLAYER_HEIGHT_INCHES',
       'PLAYER_WEIGHT', 'FGM_PAINT', 'FGM_2PT_MR', 'PW_MIN', 'PW_GAMES',
       'PW_FG3A', 'PW_FG2A', 'PW_OREB', 'PW_DREB', 'PW_ASTS', 'PW_TOV',
       'PW_STL', 'PW_BLK', 'PW_POS', 'PW_PF', 'PW_PFD', 'PW_PTS_PAINT',
       'PW_PTS_2PT_MR', 'PW_PIE_AVG', 'PW_USG_PCT_AVG', 'GAME_DATE_OBJ'],
      dtype='object')

In [541]:
plyr_box_stats_df['GAME_DATE_OBJ']

0    2022-04-10
1    2022-04-08
2    2022-04-06
3    2022-04-03
4    2022-04-01
        ...    
60   2021-10-29
61   2021-10-28
62   2021-10-26
63   2021-10-23
64   2021-10-21
Name: GAME_DATE_OBJ, Length: 65, dtype: datetime64[ns]

In [557]:
player_team_dates = plyr_box_stats_df[['TEAM_ABBREVIATION','GAME_DATE_OBJ']]

In [563]:
player_teams = player_team_dates['TEAM_ABBREVIATION'].unique()
player_teams

array(['DAL'], dtype=object)

In [566]:
team_schedule = team_dates_dict.get('DAL')
team_schedule


,Unnamed: 0,typeSeason,slugSeason,idGame,dateGame,slugMatchup,slugTeamWinner,slugTeamLoser,numberGameDay
13,14,Regular Season,2021-22,22100014,2021-10-21,DAL @ ATL,ATL,DAL,1
28,29,Regular Season,2021-22,22100029,2021-10-23,TOR vs. DAL,DAL,TOR,3
51,52,Regular Season,2021-22,22100052,2021-10-26,DAL vs. HOU,DAL,HOU,3
68,69,Regular Season,2021-22,22100069,2021-10-28,SAS @ DAL,DAL,SAS,5
74,75,Regular Season,2021-22,22100075,2021-10-29,DEN vs. DAL,DEN,DAL,5
...,...,...,...,...,...,...,...,...,...
1151,1152,Regular Season,2021-22,22101152,2022-04-01,DAL @ WAS,WAS,DAL,2
1166,1167,Regular Season,2021-22,22101167,2022-04-03,MIL vs. DAL,DAL,MIL,2
1189,1190,Regular Season,2021-22,22101190,2022-04-06,DAL @ DET,DAL,DET,1
1208,1209,Regular Season,2021-22,22101209,2022-04-08,POR @ DAL,DAL,POR,7


In [ ]:
team_dates = team_schedule[]

In [723]:
for team in player_teams:
    player_games = plyr_box_stats_df.loc[plyr_box_stats_df['TEAM_ABBREVIATION']==team]
    # reverse player_games since team schedule first -> last
    player_games.sort_values(by='GAME_DATE_OBJ', ascending=True, inplace = True)
    player_dates = player_games['GAME_DATE_OBJ']
    max_date = max(player_dates)
    min_date = min(player_dates)
    team_schedule = team_dates_dict.get(team)
    team_dates = team_schedule['dateGame_OBJ']
    missed_dates = np.setdiff1d(team_dates,player_dates, assume_unique=False)
    team_max_date = max(team_dates)
    team_min_date = min(team_dates)
    played_schedule = []
    for date in team_dates:
        if date in missed_dates:
            played_schedule.append(False)
        else:
            played_schedule.append(True)
    
    # create table of team schedule with player in game
    temp_df = pd.DataFrame({'GAME_DATE_OBJ':team_dates,
                           'PLAYED_GAME':played_schedule,})
    temp_df.reset_index(drop=True, inplace=True)

    # create column flagging player out next game
    out_next_game = []
    for index, current_date in temp_df.iterrows():
        if index == len(temp_df) - 1:
            out_next_game.append(0)
        else:
            next_date = temp_df.iloc[index + 1]
            if next_date['PLAYED_GAME']:
                out_next_game.append(0)
            else:
                out_next_game.append(1)
    temp_df.loc[:,'OUT_NEXT_GAME']  = out_next_game

    # get OUT_NEXT_GAME based on player game logs
    misc_data = temp_df.loc[temp_df['GAME_DATE_OBJ'].isin(player_dates)]

    days_last_injury = []
    date_last_injury = last_injury_date
    career_injuries = []
    career_injuries_count = num_career_injuries
    for idx, row in misc_data.iterrows():
        if row['OUT_NEXT_GAME'] == 1:
            career_injuries_count = career_injuries_count + 1
            date_last_injury = row['GAME_DATE_OBJ']
    
        time_delta = row['GAME_DATE_OBJ'] - date_last_injury
        days_last_injury.append(time_delta.days)
        career_injuries.append(career_injuries_count)

    misc_data.loc[:,'CAREER_INJURIES'] = career_injuries
    misc_data.loc[:,'DAYS_LAST_INJURY'] = days_last_injury

        
    # reverse the list again because player game logs last -> first
    misc_data_rev = misc_data.iloc[::-1]
    misc_data_rev.reset_index(drop=True, inplace=True)
    
    plyr_box_stats_df.loc[:,'OUT_NEXT_GAME'] = misc_data_rev['OUT_NEXT_GAME']
    plyr_box_stats_df.loc[:,'CAREER_INJURIES'] = misc_data_rev['CAREER_INJURIES']
    plyr_box_stats_df.loc[:,'DAYS_LAST_INJURY'] = misc_data_rev['DAYS_LAST_INJURY']
    


C:\Users\Reed\AppData\Local\Temp\ipykernel_7540\2300763361.py:54: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  misc_data.loc[:,'CAREER_INJURIES'] = career_injuries
C:\Users\Reed\AppData\Local\Temp\ipykernel_7540\2300763361.py:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  misc_data.loc[:,'DAYS_LAST_INJURY'] = days_last_injury


In [724]:
print(plyr_box_stats_df)

   SEASON_YEAR  PLAYER_ID  PLAYER_NAME     TEAM_ID TEAM_ABBREVIATION  \
0      2021-22    1629029  Luka Doncic  1610612742               DAL   
1      2021-22    1629029  Luka Doncic  1610612742               DAL   
2      2021-22    1629029  Luka Doncic  1610612742               DAL   
3      2021-22    1629029  Luka Doncic  1610612742               DAL   
4      2021-22    1629029  Luka Doncic  1610612742               DAL   
..         ...        ...          ...         ...               ...   
60     2021-22    1629029  Luka Doncic  1610612742               DAL   
61     2021-22    1629029  Luka Doncic  1610612742               DAL   
62     2021-22    1629029  Luka Doncic  1610612742               DAL   
63     2021-22    1629029  Luka Doncic  1610612742               DAL   
64     2021-22    1629029  Luka Doncic  1610612742               DAL   

           TEAM_NAME     GAME_ID            GAME_DATE      MATCHUP WL  ...  \
0   Dallas Mavericks  0022101219  2022-04-10T00:00:00  DA

In [720]:
for index, xgame in plyr_box_stats_df.iterrows():
    game_date = xgame['GAME_DATE_OBJ']
    out_next_game = xgame['OUT_NEXT_GAME']
    print("Date: "+ str(game_date) + ", out next game: " + str(out_next_game))

Date: 2022-04-10 00:00:00, out next game: 0
Date: 2022-04-08 00:00:00, out next game: 0
Date: 2022-04-06 00:00:00, out next game: 0
Date: 2022-04-03 00:00:00, out next game: 0
Date: 2022-04-01 00:00:00, out next game: 0
Date: 2022-03-30 00:00:00, out next game: 0
Date: 2022-03-29 00:00:00, out next game: 0
Date: 2022-03-27 00:00:00, out next game: 0
Date: 2022-03-25 00:00:00, out next game: 0
Date: 2022-03-21 00:00:00, out next game: 1
Date: 2022-03-19 00:00:00, out next game: 0
Date: 2022-03-18 00:00:00, out next game: 0
Date: 2022-03-16 00:00:00, out next game: 0
Date: 2022-03-13 00:00:00, out next game: 0
Date: 2022-03-11 00:00:00, out next game: 0
Date: 2022-03-09 00:00:00, out next game: 0
Date: 2022-03-07 00:00:00, out next game: 0
Date: 2022-03-03 00:00:00, out next game: 1
Date: 2022-03-01 00:00:00, out next game: 0
Date: 2022-02-27 00:00:00, out next game: 0
Date: 2022-02-25 00:00:00, out next game: 0
Date: 2022-02-17 00:00:00, out next game: 0
Date: 2022-02-15 00:00:00, out n